# Localisation et détection d'objet

Dans ce TP, nous allons mettre en pratique certaines des méthodes présentées en cours pour localiser des objets dans une image.

En localisation et détection, on cherche à déterminer la position d'un objet, ainsi que sa classe, sous la forme d'une boîte englobante de largeur $b_w$ et hauteur $b_h$, et dont le centre a pour coordonnées le point $(b_x, b_y)$. 

<center> <img src="https://drive.google.com/uc?id=1_jHHv6ZDe-3Xz25jIZ6o177laBEfmMRR" style="width:500;height:300px;"></center>
<caption><center> Figure 1: Modèle de boîte englobante utilisé pour la localisation </center></caption>

Le problème de localisation considère qu'un seul objet est présent sur l'image, alors que le problème de détection cherche à déterminer l'ensemble des objets présents sur l'image.



Pour commencer, récupérez les images de la base de données :


In [ ]:
!git clone https://github.com/axelcarlier/wildlife.git

La base de données comporte 4 classes, pour les 4 animaux suivants : buffle, éléphant, rhinocéros et zèbre.

<center> <img src="https://drive.google.com/uc?id=1sej2InBiQDEmpk2RA7S2dLRsJvF_UszE" width=200>
<img src="https://drive.google.com/uc?id=1K8cO4plzVIXO1MC4mkRcFIFjgspHtOIm" width=200>
<img src="https://drive.google.com/uc?id=15pkHpPW_VR1joOyPryS1pavOTqmoNR88" width=200>
<img src="https://drive.google.com/uc?id=19zHZn-A_j8Sx0G3V2SwR_YNKF0dGI1NA" width=200></center>
<caption><center> Figure 2: Exemples d'images de la base de données </center></caption>

# Localisation et classification d'objet

Dans cette partie, nous allons nous concentrer sur le problème de la localisation d'un seul objet par classe. Pour le cas où il y aurait plusieurs objets sur la même image, nous considérerons uniquement l'objet dont la boîte englobante occupe la plus grande surface sur l'image.



## Préparation des données

La fonction ci-dessous permet de charger les données et les formater pour la classification. Prenez le temps de regarder un peu le format des labels $y$.

In [ ]:
import PIL
from PIL import Image
import os, sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.utils import np_utils

# La base de données contient 376 images de chacune des 4 classes
DATASET_SIZE = 376*4
# Nous choisissons ici la dimension dans laquelle nous allons redimensionner les images
# 64x64 est une dimension assez faible mais qui nous permettra d'avoir des expériences plus rapides
# 128x128 ou 256x256 permettraient d'avoir de meilleurs résultats mais au prix de plusieurs heures d'entraînement
IMAGE_SIZE = 64

def load_data_localization(image_size):
  # Chemin vers la base de données
  ds_path = "./wildlife/"
  # Chemins vers les données des 4 différentes classes
  paths = [ds_path + "buffalo/", ds_path + "elephant/", ds_path + "rhino/", ds_path + "zebra/"]
  # Indice d'ajout de données dans les variables x et y 
  i = 0
  # Préparation des structures de données pour x et y
  x = np.zeros((DATASET_SIZE, image_size, image_size, 3))
  y = np.empty((DATASET_SIZE, 9)) # 9 = 1 + 4 + 4 : présence / boîte englobante / classes
    
  # Parcours des chemins de chacune des classes
  for path in paths:

    # Parcours des fichiers (classés) du répertoire
    dirs = os.listdir(path)
    dirs.sort()

    for item in dirs:
      #print(path+item)
      if os.path.isfile(path + item):
        # Extraction de l'extension du fichier 
        extension =item.split(".")[1]

        if extension=="jpg" or extension=="JPG":
          # Image : on va remplir la variable x
          # Lecture de l'image
          img = Image.open(path + item)
          # Mise à l'échelle de l'image
          img = img.resize((image_size,image_size), Image.ANTIALIAS)
          # Remplissage de la variable x
          x[i] = np.asarray(img)

        elif extension=="txt":
          # Fichier Texte : coordonnées de boîtes englobantes pour remplir y
          labels = open(path + item, "r")
          # Récupération des lignes du fichier texte
          labels= labels.read().split('\n')
          # Si la dernière ligne est vide, la supprimer 
          if labels[-1]=="":
            del labels[-1]

          # Indice de la boîte englobante de surface maximale
          j_max = 0
          if len(labels) > 1:
            aire_max = 0 # Surface de la boîte englobante de surface maximale
            # Parcours des boîtes englobantes des objets présents sur l'image
            for j in range(len(labels)):
              # Calcul de l'aire de la boîte englobante courante
              aire = float(labels[j].split()[3]) * float(labels[j].split()[4])
              # Mise à jour de la boîte englobante de surface maximale, si nécessaire
              if aire > aire_max:
                aire_max = aire
                j_max = j    

          # Un objet est présent sur l'image (presence = 1)
          presence = np.array([1], dtype="i")
          # "One-hot vector" représentant les probabilités de classe
          classes = np_utils.to_categorical(labels[j_max].split()[0], num_classes=4)
          # Coordonnées de la boîte englobante de surface maximale
          coordonnees = np.array(labels[j_max].split()[1:], dtype="f")
          # Remplissage de la variable y
          y[i, 0] = presence
          y[i, 1:5] = coordonnees
          y[i, 5:] = classes
          
          #plt.imshow()
          i = i + 1
        else:
          print("extension trouvée : ", extension)

  return x, y

x,y = load_data_localization(IMAGE_SIZE)



Séparation des données en ensembles d'entraînement et de validation

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=42)

# Pour améliorer l'entraînement, on peut centrer-réduire les coordonnées des bounding boxes
y_std = np.std(y_train, axis=0)
y_mean = np.mean(y_train, axis=0)
y_train[...,1:5] = (y_train[...,1:5] - y_mean[1:5])/y_std[1:5]
y_val[...,1:5] = (y_val[...,1:5] - y_mean[1:5])/y_std[1:5]

# Et normaliser les valeurs de couleur
x_train = x_train/255
x_val = x_val/255

## Fonctions utiles

Calcul du coefficient de Jaccard (intersection sur union) entre boîtes englobantes réelles et prédites.

In [ ]:
def compute_iou(y_true, y_pred):
  ### "Dénormalisation" des coordonnées des boîtes englobantes
  pred_box_xy = y_pred[..., 0:2]* y_std[0:2] + y_mean[0:2]
  true_box_xy = y_true[..., 0:2]* y_std[0:2] + y_mean[0:2]

  ### "Dénormalisation" des largeur et hauteur des boîtes englobantes
  pred_box_wh = y_pred[..., 2:4] * y_std[2:4] + y_mean[2:4]
  true_box_wh = y_true[..., 2:4] * y_std[2:4] + y_mean[2:4]
    
  # Calcul des coordonnées minimales et maximales des boiptes englobantes réelles
  true_wh_half = true_box_wh / 2.
  true_mins    = true_box_xy - true_wh_half
  true_maxes   = true_box_xy + true_wh_half
  
  # Calcul des coordonnées minimales et maximales des boiptes englobantes prédites
  pred_wh_half = pred_box_wh / 2.
  pred_mins    = pred_box_xy - pred_wh_half
  pred_maxes   = pred_box_xy + pred_wh_half       
  
  # Détermination de l'intersection des boîtes englobantes
  intersect_mins  = tf.maximum(pred_mins,  true_mins)
  intersect_maxes = tf.minimum(pred_maxes, true_maxes)
  intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
  intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
  
  # Aire des boîtes englobantes prédites et réelles
  true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
  pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

  # Aire de l'union des boîtes prédites et réelles
  union_areas = pred_areas + true_areas - intersect_areas

  iou_scores  = tf.truediv(intersect_areas, union_areas)
  return iou_scores

def iou():
  def iou_metrics(y_true, y_pred):
    return compute_iou(y_true, y_pred)
  iou_metrics.__name__= "IoU"
  return iou_metrics

Visualisation des données et labels

In [ ]:
# Si seuls x et y sont indiqués, on tire au hasard un numéro d'image et on affiche le label y associé  à l'image
# Si un 2e y, nommé y_pred, est indiqué, alors les deux labels sont affichés côte à côte, afin de pouvoir les comparer
# Enfin on peut également indiquer l'id de l'image que l'on souhaite visualiser.
def print_data_localisation(x, y, y_pred=[], id=None, image_size=IMAGE_SIZE):
  if id==None:
    # Tirage aléatoire d'une image dans la base
    num_img = np.random.randint(x.shape[0]-1)
  else:
    num_img = id

  img = x[num_img]
  lab = y[num_img]

  colors = ["blue", "yellow", "red", "orange"] # Différentes couleurs pour les différentes classes
  classes = ["Buffalo", "Elephant", "Rhino", "Zebra"]

  if np.any(y_pred):
    plt.subplot(1, 2, 1)

  # Affichage de l'image
  plt.imshow(img)
  # Détermination de la classe
  class_id = np.argmax(lab[5:])

  # Détermination des coordonnées de la boîte englobante dans le repère image
  ax = (lab[1]*y_std[1] + y_mean[1]) * image_size
  ay = (lab[2]*y_std[2] + y_mean[2]) * image_size
  width = (lab[3]*y_std[3] + y_mean[3]) * image_size
  height = (lab[4]*y_std[4] + y_mean[4]) * image_size
  #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
  # Détermination des extrema de la boîte englobante
  p_x = [ax-width/2, ax+width/2]
  p_y = [ay-height/2, ay+height/2]
  # Affichage de la boîte englobante, dans la bonne couleur
  plt.plot([p_x[0], p_x[0]],p_y,color=colors[class_id])
  plt.plot([p_x[1], p_x[1]],p_y,color=colors[class_id])
  plt.plot(p_x,[p_y[0],p_y[0]],color=colors[class_id])
  plt.plot(p_x,[p_y[1],p_y[1]],color=colors[class_id])
  plt.title("Vérité Terrain : Image {} - {}".format(num_img, classes[class_id]))

  if np.any(y_pred):
    plt.subplot(1, 2, 2)
    # Affichage de l'image
    plt.imshow(img)
    lab = y_pred[num_img]
    # Détermination de la classe
    class_id = np.argmax(lab[5:])

    # Détermination des coordonnées de la boîte englobante dans le repère image
    ax = (lab[1]*y_std[1] + y_mean[1]) * image_size
    ay = (lab[2]*y_std[2] + y_mean[2]) * image_size
    width = (lab[3]*y_std[3] + y_mean[3]) * image_size
    height = (lab[4]*y_std[4] + y_mean[4]) * image_size
    #print("x: {}, y: {}, w: {}, h:{}".format(ax,ay,width, height))
    # Détermination des extrema de la boîte englobante
    p_x = [ax-width/2, ax+width/2]
    p_y = [ay-height/2, ay+height/2]
    # Affichage de la boîte englobante, dans la bonne couleur
    plt.plot([p_x[0], p_x[0]],p_y,color=colors[class_id])
    plt.plot([p_x[1], p_x[1]],p_y,color=colors[class_id])
    plt.plot(p_x,[p_y[0],p_y[0]],color=colors[class_id])
    plt.plot(p_x,[p_y[1],p_y[1]],color=colors[class_id])
    plt.title("Prédiction : Image {} - {}".format(num_img, classes[class_id]))

  plt.show()

for i in range(100):#x.shape[0]):
    print_data_localisation(x_train, y_train, image_size=IMAGE_SIZE, id=i)


In [ ]:
def plot_training_analysis(history, metric='loss'):    

  loss = history.history[metric]
  val_loss = history.history['val_' + metric]

  epochs = range(len(loss))

  plt.plot(epochs, loss, 'b', linestyle="--",label='Training ' + metric)
  plt.plot(epochs, val_loss, 'g', label='Validation ' + metric)
  plt.title('Training and validation ' + metric)
  plt.legend()

  plt.show()

## Travail à faire



<center> <img src="https://drive.google.com/uc?id=1YzCZe4pgnjJDVGklAaCHZ7HhlPJsadg9" width=500></center>
<caption><center> Figure 3: Illustration de l'architecture du réseau à construire.  </center></caption>

Complétez les codes qui vous sont fournis pour obtenir un algorithme de localisation. 
Vous pouvez utiliser n'importe quelle base convolutive de votre choix, en revanche vous devrez porter une attention particulière à la couche de sortie.

Vous allez en fait produire 3 sorties différentes : une caractérisant la présence d'un objet, une autre fournissant les coordonnées de la boîte englobante, et enfin une dernière effectuant la classification.

<center> <img src="https://drive.google.com/uc?id=1bnh8zU7Os-w-5TT8hV4xDoThKQc-Ywc2" width=500></center>
<caption><center> Figure 4: Illustration des fonctions de coût à utiliser pour l'entraînement. </center></caption>

In [ ]:
import keras
from keras import layers
from keras import models
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from keras.models import Model, Sequential

def create_model_localisation(input_shape=(64, 64, 3)):

  input_layer = Input(shape=input_shape)


  conv1 = Conv2D(...)(input_layer)
  # Convolutions, Pooling, Dense,  à vous de former votre réseau
  # Votre dernière couche doit mettre à jour une variable x, réutilisée dans les couches de sortie ci-dessous
  x = ...

  output_p = Dense(..., activation=..., name='p')(x)   # Sortie caractérisant la présence d'un objet
  output_coord = Dense(..., activation=..., name='coord')(x) # Sortie caractérisant les coordonnées de boîte englobante
  output_class = Dense(..., activation=..., name='classes')(x) # Sortie caractérisant les probabilités de classe
  
  output= [output_p, output_coord, output_class]
  model = Model(input_layer, output)

  return model

Pour entraîner votre réseau, vous allez donc devoir associer une fonction de coût à chacune des sorties du réseau. La fonction de coût totale sera la somme des trois fonctions de coût précédemment définies, pondérées par des poids définis dans la variable *loss_weights*.

**Prenez le temps de tester différentes valeurs de *loss_weights* en fonction de l'évolution des métriques que vous observerez pendant l'entraînement.**

In [ ]:
from keras.optimizers import Adam

import tensorflow as tf

batch_size=16
model = create_model_localisation()
opt = Adam(learning_rate=3e-4)  

# Ici mettre, dans l'ordre, les fonctions de coût associées à chacune des sorties
loss=[..., ..., ...]
# On va associer une métrique à chaque sortie : l'accuracy pour les deux classifications, 
# et l'IoU définie plus tôt pour la qualité des boîtes englobantes. 
metrics=[ ['accuracy'], [iou()], ['accuracy']]
loss_weights = [1, 5, 1]

model.compile(loss=loss,
              optimizer=opt,
              metrics=metrics,
              loss_weights=loss_weights
              )

history = model.fit(x_train, [y_train[:,0], y_train[:,1:5], y_train[:,5:9]],
              epochs=30,
              batch_size=batch_size,            
              validation_data=(x_val, [y_val[:,0], y_val[:,1:5], y_val[:,5:9]]))



In [ ]:
# Analyse des résultats : courbes d'évolution de la fonction de perte, et de l'IoU des boîtes englobantes ainsi que de l'accuracy des classes prédites
plot_training_analysis(history, metric='loss')
plot_training_analysis(history, metric='coord_IoU')
plot_training_analysis(history, metric='classes_accuracy')

# Prédiction des données de validation
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)
for i in range(y_pred.shape[0]):
  y_pred[i, 0] = y_pred_presence[i]
  y_pred[i, 1:5] = y_pred_coords[i]
  y_pred[i, 5:9] = y_pred_classes[i]

# Affichage des résultats sur plusieurs images
print_data_localisation(x_val, y_val, y_pred = y_pred, id=2, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=3, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=7, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=25, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=16, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=18, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=24, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=15, image_size=IMAGE_SIZE)

En pratique, il est délicat de trouver une bonne combinaison des fonctions de perte tel que vous l'avez fait sur les cellules précédentes. L'entropie croisée et l'erreur quadratique moyenne donnent des valeurs trop différentes pour être combinables efficacement.

Une variante, peut-être plus simple à faire fonctionner, est d'utiliser uniquement l'erreur quadratique moyenne comme perte pour toutes les sorties. C'est cette variante qui est implémentée dans l'algorithme YOLO, dont nous implémenterons une variante dans le prochain TP.
Testez cette solution ci-dessous. Comme sur l'exercice précédent, n'hésitez pas à faire varier le poids des différents éléments de la fonction de coût.

In [ ]:
from keras.optimizers import Adam

import tensorflow as tf

batch_size=16
model = create_model_localisation()
opt = Adam(learning_rate=3e-4)  

loss = [..., ..., ...]
metrics =[ ['accuracy'], [iou()], ['accuracy']]
loss_weights = [...]

model.compile(loss=loss,
              optimizer=opt,
              metrics=metrics,
              loss_weights=loss_weights
              )
history = model.fit(x_train, [y_train[:,0], y_train[:,1:5], y_train[:,5:9]],
              epochs=30,
              batch_size=batch_size,            
              validation_data=(x_val, [y_val[:,0], y_val[:,1:5], y_val[:,5:9]]))


In [ ]:
# Analyse des résultats : courbes d'évolution de la fonction de perte, et de l'IoU des boîtes englobantes ainsi que de l'accuracy des classes prédites
plot_training_analysis(history, metric='loss')
plot_training_analysis(history, metric='coord_IoU')
plot_training_analysis(history, metric='classes_accuracy')

# Prédiction des données de validation
y_pred_presence, y_pred_coords, y_pred_classes = model.predict(x_val)
y_pred = np.zeros(y_val.shape)
for i in range(y_pred.shape[0]):
  y_pred[i, 0] = y_pred_presence[i]
  y_pred[i, 1:5] = y_pred_coords[i]
  y_pred[i, 5:9] = y_pred_classes[i]

print_data_localisation(x_val, y_val, y_pred = y_pred, id=2, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=3, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=7, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=25, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=16, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=18, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=24, image_size=IMAGE_SIZE)
print_data_localisation(x_val, y_val, y_pred = y_pred, id=15, image_size=IMAGE_SIZE)


Compte-tenu de la taille réduite de la base de données, les résultats ne sont pas mal du tout ! On observe quelques confusions entre certaines classes mais les prédictions sont souvent intéressantes.

Il devrait cependant subsister un fort surapprentissage à ce stade. Comme nous l'avons vu dans de précédents TPs, vous avez plusieurs possibilités qui s'offrent à vous pour le corriger : 


*   Régularisation par *weight decay* (utilisation de *kernel_regularizer* sur les couches de votre réseau)
*   Augmentation de la base de données. Vous pouvez pour cela vous appuyer sur l'exemple fourni en TP précédent, avec une classe *Sequence* et l'utilisation de la librairie *Albumentation*.
*   Utilisation de *transfer learning* : partant d'un réseau entraîné sur ImageNet (qui contient de nombreuses classes d'animaux), vous bénéficieriez de filtres très généraux qui aiderait à limiter le surapprentissage. 



### Code fourni pour vous aider à mettre en place l'augmentation de données

On met à jour la version d'Albumentations.ai pour bénéficier des dernières fonctionnalités :

In [ ]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

In [ ]:
import cv2 as cv
from albumentations import (Compose, RandomBrightness, RandomContrast, RandomGamma, ShiftScaleRotate, RandomSizedBBoxSafeCrop)
import albumentations as A

AUGMENTATIONS_TRAIN = Compose([
    ShiftScaleRotate(p=0.5),
    RandomContrast(limit=0.2, p=0.5),
    RandomGamma(gamma_limit=(80, 120), p=0.5),
    RandomBrightness(limit=0.2, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [ ]:
from tensorflow.python.keras.utils.data_utils import Sequence

class WildLifeSequence(Sequence):
    # Initialisation de la séquence avec différents paramètres
    def __init__(self, x_set, y_set, batch_size,augmentations):
        self.x, self.y = x_set, y_set
        self.classes = ["Buffalo", "Elephant", "Rhino", "Zebra"]
        self.batch_size = batch_size
        self.augment = augmentations
        self.indices1 = np.arange(x_set.shape[0]) 
        np.random.shuffle(self.indices1) # Les indices permettent d'accéder
        # aux données et sont randomisés à chaque epoch pour varier la composition
        # des batches au cours de l'entraînement

    # Fonction calculant le nombre de pas de descente du gradient par epoch
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    # Il y a des problèmes d'arrondi dans les conversions de boîtes englobantes 
    # internes à la librairie Albumentations
    # Pour les contourner, si les boîtes sont trop proches des bords, on les érode un peu
    def erode_bounding_box(self, box):
        epsilon = 0.01
        
        xmin = max(box[0] - box[2]/2, epsilon)
        ymin = max(box[1] - box[3]/2, epsilon)
        xmax = min(box[0] + box[2]/2, 1-epsilon)
        ymax = min(box[1] + box[3]/2, 1-epsilon)
        
        cx = xmin + (xmax - xmin)/2
        cy = ymin + (ymax - ymin)/2
        width = xmax - xmin
        height = ymax - ymin
        
        return np.array([cx, cy, width, height])
    
    # Application de l'augmentation de données à chaque image du batch et aux
    # boîtes englobantes associées
    def apply_augmentation(self, bx, by):

        batch_x = np.zeros((bx.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))
        batch_y = by
        
        # Pour chaque image du batch
        for i in range(len(bx)):
            bboxes = []
            box = by[i,1:5]
            # Dénormalisation des coordonnées de boites englobantes
            box[0] = (box[0]*y_std[1] + y_mean[1])
            box[1] = (box[1]*y_std[2] + y_mean[2])
            box[2] = (box[2]*y_std[3] + y_mean[3])
            box[3] = (box[3]*y_std[4] + y_mean[4])
            box = self.erode_bounding_box(box)
            bboxes.append(box)
            
            class_labels = []
            class_id = np.argmax(by[i, 5:])
            class_labels.append(self.classes[class_id])

            img = bx[i]

            # Application de l'augmentation à l'image et aux masques
            transformed = self.augment(image=img.astype('float32'), bboxes=bboxes, class_labels=class_labels)
            batch_x[i] = transformed['image']
            batch_y_transformed = transformed['bboxes']
                
            # Renormalisation des coordonnées de boîte englobante transformée
            batch_y[i, 1] = (batch_y_transformed[0][0] - y_mean[1])/y_std[1]
            batch_y[i, 2] = (batch_y_transformed[0][1] - y_mean[2])/y_std[2]
            batch_y[i, 3] = (batch_y_transformed[0][2] - y_mean[3])/y_std[3]
            batch_y[i, 4] = (batch_y_transformed[0][3] - y_mean[4])/y_std[4]

        return batch_x, batch_y

    # Fonction appelée à chaque nouveau batch : sélection et augmentation des données
    def __getitem__(self, idx):
        batch_x = self.x[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_y = self.y[self.indices1[idx * self.batch_size:(idx + 1) * self.batch_size]]
        batch_x, batch_y = self.apply_augmentation(batch_x, batch_y)
        
        batch_y = np.array(batch_y)
        return np.array(batch_x), [batch_y[:,0], batch_y[:,1:5], batch_y[:,5:9]]

    # Fonction appelée à la fin d'un epoch ; on randomise les indices d'accès aux données
    def on_epoch_end(self):
        np.random.shuffle(self.indices1)
        


In [ ]:
# Instanciation d'une Sequence
train_gen = WildLifeSequence(x_train, y_train, 16, augmentations=AUGMENTATIONS_TRAIN)

# Pour tester la séquence, nous sélectionnons les éléments du premier batch et les affichons
batch_x, batch_y = train_gen.__getitem__(0)

y_batch = np.zeros((batch_y[0].shape[0],9))

for i in range(batch_y[0].shape[0]):
  y_batch[i, 0] = batch_y[0][i]
  y_batch[i, 1:5] = batch_y[1][i]
  y_batch[i, 5:9] = batch_y[2][i]

print_data_localisation(batch_x, y_batch, image_size=IMAGE_SIZE)